ML2 Spiking Network Exercise with PyNN & NEST
==============

In this exercise, we will use PyNN & NEST to learn visual event-based features with STDP.

    
Tools
--------

NEST is a popular spiking neural network simulator. 
PyNN is a wrapper around nest, and other neural network simulator (including neuromorphic hardware).


Installation
-------------

We will install all the required libraries in your ~/.local folder, so make sure your PYTHONPATH contains this folder:

```bash
export PYTHONPATH=$PYTHONPATH:~/.local/lib/x86_64-linux-gnu/python2.7/site-packages:~/.local/lib/python2.7/site-packages
```

Download, compile and install NEST version 2.12.0:

```bash
git clone git@github.com:nest/nest-simulator.git
cd nest-simulator
git checkout v2.12.0
mkdir build 
cd build
cmake -DCMAKE_INSTALL_PREFIX:PATH=~/.local ..
make install
```
    
The other libraries can be installed with pip, except for python-quantities:

```bash
git clone git://github.com/python-quantities/python-quantities.git
cd python-quantities
python setup.py install --user
pip install --user -r requirements.txt
```

# Make sure you can import everything we need

In [ ]:
import pyNN.nest as sim
import numpy as np
from scipy import ndimage
import matplotlib.pyplot as plt
sim.setup()

In [ ]:
import PIL.Image
from cStringIO import StringIO
import IPython.display
import numpy as np
def showarray(a, fmt='png'):
    a = np.uint8(a)
    f = StringIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))


-------------------------


# Introduction to the exercise

In this exercise, we will learn visual features over some samples of the MNIST dataset with spiking neural network.
The features will be learned in an unsupervised way with STDP.
By following this exercise, you will have fun & rehearse:
* The STDP learning rule
* Winner-take-all
* Encoding visual stimuli

By exploring a little further, you can also experiment by yourself:
* Dendritic delays
* Assossiative learning
* Dropouts (stochastic synapses)
* Noise
* ...

In the whole exercise, we have a feedforward 2-layers network (input and output).
The whole exercise is similar to __P. U. Diehl and M. Cook, “Unsupervised learning of digit recognition using spike-timing-dependent plasticity.,” Front. Comput. Neurosci., vol. 9, no. August, p. 99, 2015.__

## We load our stimuli: some MNIST samples


In [ ]:
stimuli_filenames = [
    "m0.png",
    "m1.png",
    "m2.png",
    "m4.png",
    "m5.png",
    "m9.png"
]
stimuli = [ ndimage.imread('images/'+stimulus) for stimulus in stimuli_filenames]
for stimulus in stimuli:
    showarray(stimulus)
input_shape = stimuli[0].shape

## We define some utility functions

* __simulate_network()__: set the stimuli one by one to the input neurons and simulate with pauses inbetween
* __create_network()__: create input and output population; but **do not create the connection: this is your job!**
* __plot_spiketrain()__: plot the spiketrain of the output neurons with the lines indicating the stimuli
* __plot_weights()__: plot the weights of the projection assuming an array of shape input_shape

In [ ]:
%matplotlib inline
stim_exposure = 200
stim_pause = 10
epsilon = 0.001
def simulate_network(stimuli, input_layer):
    for stim in stimuli:
        # For some reasons, I had issues with setting the rate of input Poisson neurons.
        # So instead, we can use LIF neurons with constant input current
        if isinstance(input_layer.celltype, sim.SpikeSourcePoisson):
            input_layer.set(rate=(stim.flatten() / 255. * 60. + epsilon).tolist())
            sim.run(stim_exposure)
            input_layer.set(rate=0.00001)
            sim.run(stim_pause)
        elif isinstance(input_layer.celltype, sim.IF_curr_exp):
            input_layer.set(i_offset=(stim.flatten() / 255. * 4.).tolist())
            sim.run(stim_exposure)
            input_layer.set(i_offset=0.)
            sim.run(stim_pause)

def create_network(input_shape, n_output):
    sim.setup()
    cell_parameters = {
        "tau_m": 10.0,  # membrane time constant (ms) ???
        "v_thresh": -50.0,  # spike threshold (mV)
        "v_reset": -60.0,  # reset potential after a spike (mV)
        "v_rest": -60.0,  # resting membrane potential (mV)
        "cm": 1.0,  # membrane capacity (nF)
        "tau_refrac": 2,  # (relative) refractory period (ms)
    }

    input_pop = sim.Population(np.prod(input_shape), 
                               #sim.SpikeSourcePoisson(rate=0.),
                               sim.IF_curr_exp(),
                               label="input",
                               initial_values={
                                    "v": cell_parameters["v_reset"]
                                })
    input_pop.record('spikes') 
    output_pop = sim.Population(n_output,
                                sim.IF_curr_exp(**cell_parameters),
                                label="output",
                                initial_values={
                                    "v": cell_parameters["v_reset"]
                                })
    output_pop.record('spikes') 
    return (input_pop, output_pop)

def plot_spiketrain(layer, n_stimuli):
    # get_data() get the recorded variables (here only spikes) in a Neo object
    spiketrains = layer.get_data().segments[0].spiketrains
    plt.figure(figsize=(11,7))
    ax = plt.subplot(111)
    ax.set_title('{} spiketrain'.format(layer.label))
    for spiketrain in spiketrains:
        ax.plot(spiketrain,
                np.ones_like(spiketrain) * spiketrain.annotations['source_index'],
                '|', color='b', markersize='20')
    # We also plot the vertical lines indicating the start of the stimuli
    for stim_idx in range(n_stimuli):
        ax.axvline(x= stim_idx * (stim_exposure + stim_pause),
                  linewidth=3, color='r', label='Stimulus {}'.format(stim_idx))
        ax.axvline(x= stim_exposure + stim_idx * (stim_exposure + stim_pause),
                  linewidth=3, color='g', label='Pause {}'.format(stim_idx))        
    if (len(spiketrain>0)):
        ax.set_xlim([0, float(spiketrain[-1])])
    ax.set_ylabel("Neuron index")
    ax.set_xlabel("Time (ms)")
    
    
def plot_weights(proj):
    all_neurons_weights = proj.get('weight', format='array')
    for single_neuron_weights in np.rollaxis(all_neurons_weights, 1):        
        H = np.reshape(single_neuron_weights, input_shape)

        fig = plt.figure(figsize=(8, 5))

        ax = fig.add_subplot(111)
        ax.set_title('Weight matrix')
        plt.imshow(H)
        ax.set_aspect('equal')

        cax = fig.add_axes([0.12, 0.1, 0.78, 0.8])
        cax.get_xaxis().set_visible(False)
        cax.get_yaxis().set_visible(False)
        cax.patch.set_alpha(0)
        cax.set_frame_on(False)
        plt.colorbar(orientation='vertical')
        plt.show()
    

## Running the network and plotting the results

Here is what the network does when presented with the stimuli and static weight connections (not learning):

In [ ]:
input_layer, output_layer = create_network(input_shape, 10)
proj = sim.Projection(input_layer, output_layer,
                          sim.AllToAllConnector(),
                          sim.StaticSynapse(weight=0.015))
simulate_network(stimuli, input_layer)

## Input spiketrain

In [ ]:
plot_spiketrain(input_layer, len(stimuli))

## Output spiketrain

In [ ]:
plot_spiketrain(output_layer, len(stimuli))


---------------------------

# Exercise 1 - fix the projection

All the output neurons have the same spiketrain! What is going on?
If all the output spikes are the exact same, we will learn the same features with all neurons.
We need to introduce some variability in the output spiketrains.
Try to fix it in the following notebook cell by changing the Projection.

Hints:
* http://neuralensemble.org/docs/PyNN/connections.html
* http://neuralensemble.org/docs/PyNN/reference/connectors.html

If you don't have a clue, you can use use the __plot_weights()__ function to get some insights.

In [ ]:
input_layer, output_layer = create_network(input_shape, 10)
initial_weights = sim.random. # Complete here...
synapses = sim.Projection(input_layer, output_layer,
                          sim.AllToAllConnector(),
                          sim.StaticSynapse(weight=initial_weights))
simulate_network(stimuli, input_layer)
plot_spiketrain(output_layer, len(stimuli))

# Exercise 2 - Learning features

Until now, our synapses were still static.
Now, we will make them plastic and start learning.
Let's plot the current weights of output neurons:

In [ ]:
plot_weights(synapses)

Do these weights looks like meaningful features?
Not really. Let's try to learn better ones with STDP.
Change the synapse type of the projection from StaticSynapse to STDPMechanism.

Hint: http://neuralensemble.org/docs/PyNN/reference/plasticitymodels.html

You can also try to do many epochs by running the __simulate_network()__ in a loop.

--------------

If you get the error:

```bash    
BadProperty in Connect_g_g_D_D: Weight and Wmax must have same sign
```

This means that you have some negative weights in your initial weight distribution. 
Sadly, NEST does not support weights dropping below 0 during STDP. 
Therefore, make sure that your initial_weight distribution contains only positiv numbers.
Different possibilities:
* You can change the distribution you use 
* You can add pure inhibitory synapses on top of the exibithory ones, to compensate
* You can get/set the weights with a numpy array with:
    
```python
w = synapses.get('weight')
synapses.set(weight=w)
```

with __w__ being a weight matrix.

In [ ]:
input_layer, output_layer = create_network(input_shape, 10)

stdp_model = # Complete here...

synapses = sim.Projection(input_layer, output_layer, # ... and here
                         )


simulate_network(stimuli, input_layer)
plot_spiketrain(output_layer, len(stimuli))
plot_weights(synapses)

# Exercise 3 - Learning discriminative features

Do the weights look like something meaningful? 
If all the weights seem to be a "combination" of all the digits at once, we say that the features are not __discriminative__.
Indeed, when a digit is presented and the initial weights of an output neurons are high enough to make this neuron spike (natural bias, or preference), this neuron will be better at detecting this digit thanks to STDP.
However, when another digit is presented, if the same neuron also has a natural bias for this other digit, then it will learn to detect this digit too.
Therefore, its weights will be a combination of the digits it knows.

To prevent this from happening, a common technique is to use lateral inhibition, to implement a __winner-take-all__. 
The idea is, when an output neuron spike because of a natural bias to a digit, it will inhibit the other output neurons, preventing them to learn the same number.
Lateral inhibition is a connection from a layer to itself, with all the weights being some constant, except for the self connection which is 0 (the neuron does not inhibit itself).

Add a lateral inhibition on the output layer and plot the weights again.

In [ ]:
def lateral_inhib(output_layer, weight):
    n_output_neurons = len(output_layer)
    lateral_inhib = sim.Projection(output_layer, output_layer,
                                   sim.AllToAllConnector(allow_self_connections=False),
                                   sim.StaticSynapse(weight=weight, delay=0.1),
                                   receptor_type='inhibitory'
                                  )

# Going further

You can continue playing with the unsupervised learning of features.
* What if you change the STDP parameters?
* What if you add noise to the network, in the form of a Poisson generator connecting to all output neurons?
* What if use a distribution for the dendritic delays in the input/output connection?
* What about different input images?

You can also try to change the learning algorithm to a supervised one, with __associative learning__.
Until now, there is no way to say what output neuron will learn what digit.
With associative learning, you can have a teaching neuron which forces an output neuron to spike at a given frequency when a given digit is presented.